In [1]:
using Base.Math
using Dates
using CSV
using DataFrames
using LinearAlgebra
using Plots
using StaticArrays

using SatelliteToolbox
using SatelliteToolboxBase
using SatelliteToolboxPropagators
using SatelliteToolboxTransformations
using SatelliteToolboxAtmosphericModels
using SatelliteToolboxGravityModels
using SatelliteToolboxCelestialBodies

In [ ]:
using PyCall

py"""

def Plot2D(ground_lons, ground_lats):

    import numpy as np
    import matplotlib.pyplot as plt
    from cartopy import crs as ccrs

    # Mercator Projection ==========================================================================================================
    plt.figure(figsize=(10, 6))
    # ax = plt.axes(projection=ccrs.PlateCarree())
    ax = plt.axes(projection=ccrs.Orthographic())
    # ax = plt.axes(projection=ccrs.InterruptedGoodeHomolosine())
    
    ax.stock_img()
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    ax.plot(ground_lons, ground_lats, 'r-', transform=ccrs.Geodetic(), label='Ground track', zorder=1) # Orbital path

    plt.legend(loc='upper right')
    plt.title("Ground Track del Satélite")
    plt.show()

"""

In [62]:
function Propagate_SPG4(dt, simtime_minutes, epoch, sat_altitude_km, eccentricity, inclination_deg, raan_deg, arg_perigee_deg, mean_anomaly_deg, drag_coefficient)
    
    epoch_jd = datetime2julian(epoch)

    μ = 3.986004418e14
    earth_radius_km = 6378.137
    altitude = (sat_altitude_km + earth_radius_km) * 1000
    velocity_rad_s   = sqrt(μ / altitude^3)

    inclination      = deg2rad(inclination_deg)
    raan             = deg2rad(raan_deg)
    arg_perigee      = deg2rad(arg_perigee_deg)
    mean_anomaly     = deg2rad(mean_anomaly_deg)

    initializer = Propagators.init(Val(:SGP4), epoch_jd, velocity_rad_s, eccentricity, inclination, raan, arg_perigee, mean_anomaly, drag_coefficient)

    t_span = 0:dt:60*simtime_minutes

    timestamps, latitudes, longitudes, altitudes, speeds = Float64[], Float64[], Float64[], Float64[], Float64[], Float64[]
    photo_flags, comm_flags = Int[], Int[]

    # Ground station coordinates
    gs_lat_deg = 25.67507
    gs_lon_deg = -100.31847
    gs_lat_rad = deg2rad(gs_lat_deg)
    gs_lon_rad = deg2rad(gs_lon_deg)
    gs_ecef = geodetic_to_ecef(gs_lat_rad, gs_lon_rad, 0.0)

    eop_data = fetch_iers_eop(Val(:IAU1980))

    for (i, t) in enumerate(t_span)

        r_eci, v_eci = Propagators.step!(initializer, dt)

        # lat_rad, lon_rad, alt_km = ecef_to_geodetic(r)
        
        jd = epoch_jd + t / 86400.0
        dcm = r_eci_to_ecef(DCM, GCRF(), ITRF(), jd, eop_data)
        r_ecef = dcm * r_eci
        lat_rad, lon_rad, alt_m = ecef_to_geodetic(r_ecef)

        if alt_m < 0
            break
        end
        
        # Determine photo flag
        lat_deg = rad2deg(lat_rad)
        lon_deg = rad2deg(lon_rad)
        photo_flag = (-60 <= lat_deg <= -50) ? 1 : 0
        
        # Determine communication flag
        range_vector = r_eci - gs_ecef
        range_norm = norm(range_vector)
        elevation_rad = asin((range_vector[3]) / range_norm)
        comm_flag = (rad2deg(elevation_rad) >= 10) ? 1 : 0
        
        push!(timestamps, t)
        
        push!(latitudes, lat_deg)
        push!(longitudes, lon_deg)
        push!(altitudes, alt_m/1000)
        
        push!(speeds, norm(v_eci))
        
        push!(photo_flags, photo_flag)
        push!(comm_flags, comm_flag)

    end

    return timestamps, latitudes, longitudes, altitudes, speeds, photo_flags, comm_flags

end;

In [74]:
dt = 3000
simtime_minutes = 30 * 365 * 24 * 60

epoch            = DateTime(2025, 1, 1, 0, 0, 0)
sat_altitude_km  = 650
eccentricity     = 0.0
inclination_deg  = 97.5
raan_deg         = 0
arg_perigee_deg  = 0
mean_anomaly_deg = 0
drag_coefficient = 5e-4

timestamps, latitudes, longitudes, altitudes, speeds, photo_flags, comm_flags = Propagate_SPG4(
    dt, simtime_minutes, epoch, sat_altitude_km, eccentricity, inclination_deg, raan_deg, arg_perigee_deg, mean_anomaly_deg, drag_coefficient)

# py"Plot2D"(longitudes, latitudes)

([0.0, 3000.0, 6000.0, 9000.0, 12000.0, 15000.0, 18000.0, 21000.0, 24000.0, 27000.0  …  8.37606e8, 8.37609e8, 8.37612e8, 8.37615e8, 8.37618e8, 8.37621e8, 8.37624e8, 8.37627e8, 8.3763e8, 8.37633e8], [-4.322086075840353, 8.14424251837802, -12.44027404890128, 16.264448202805497, -20.545544577300056, 24.373041475832856, -28.630718578580645, 32.462238626870175, -36.687760506502194, 40.522724546461156  …  -43.654907053252245, 73.56631573698807, -72.12537124987924, 42.00346119661823, -10.921277256432955, -20.41012003704632, 51.27498472886704, -80.03309936661188, 64.83315973404247, -34.221005512702085], [78.90385191294486, -114.10506830370682, 52.81115487604167, -140.22511119361278, 26.64903921451532, -166.44116065637687, 0.35843446273125035, 167.17728221293282, -26.145350029662097, 140.52475848934296  …  103.33745209068589, -107.68872932936085, -71.53621416064401, 79.35593886890781, -118.30127964166557, 45.09075599490679, -153.80644486255179, -23.422491807851422, 25.966514246935695, -177.2484

In [ ]:
using Statistics

mean_days = 30

altitudes = collect(altitudes)
samples_per_day = round(Int, mean_days * 60 * 60 * 24 / dt)
n_days = div(length(altitudes), samples_per_day)

daily_time_vector = timestamps[1:samples_per_day:end][1:n_days]

seconds_per_year = 365.25 * 24 * 60 * 60  # includes leap year correction
daily_time_vector_years = daily_time_vector ./ seconds_per_year

altitudes_reshaped = reshape(altitudes[1:n_days * samples_per_day], samples_per_day, n_days)
daily_mean_altitudes = mean(altitudes_reshaped, dims=1)[:]

plot!(daily_time_vector_years, daily_mean_altitudes, title="Altitud", 
xlabel="Tiempo (años)", ylabel="Órbita del satélite (km)",  legend=false; dpi=400)
# savefig("orbita_fall_2.png")

"/Users/miguelcomett/Documents/Physics/IFI 🪄/8to Semestre/BII – Integración IFI/RETO/3_Propagator/orbita_fall_2.png"

In [ ]:
plot(timestamps, altitudes, title="Altitud", xlabel="Minutos (s)", ylabel="Órbita del satélite km", legend=false)

In [ ]:
using LinearAlgebra
using SatelliteToolboxTransformations

sat_altitude_km  = 650
eccentricity     = 0.0
inclination_deg  = 97.5
raan_deg         = 0
arg_perigee_deg  = 0
mean_anomaly_deg = 0

earth_radius_km = 6378.137
altitude = (sat_altitude_km + earth_radius_km) * 1000

latitude_deg = 0
longitude_deg = 0

r_ecef = geocentric_to_ecef(latitude_deg, longitude_deg, altitude)

eop_data = fetch_iers_eop(Val(:IAU1980))
epoch = DateTime(2025, 1, 1, 0, 0, 0)
epoch_jd = datetime2julian(epoch)
eci = r_ecef_to_eci(DCM, Val(:PEF), Val(:TOD), epoch_jd, eop_data)

r_eci = eci * r_ecef
rx, ry, rz = r_eci
r = [rx, ry, rz]

velx_m_s = 4000
vely_m_s = sqrt(7500^2 - velx_m_s^2)

v = [0, velx_m_s, vely_m_s]

h = cross(r, v)
h_mag = norm(h)

inclination_rad = acos(h[3] / h_mag)
inclination_deg = rad2deg(inclination_rad)

# println(r_ecef)
# println(r_eci)
# println(h)
# println(h_mag)
# println(inclination_rad)
println("inclinación: ", inclination_deg, "°")

In [ ]:
# v_ned = [velx_m_s, vely_m_s, 0.0]
# v_ecef = ned_to_ecef(v_ned, latitude_deg, longitude_deg, altitude; translate=false)
# println(v_ecef)
# sv_ecef = OrbitStateVector(epoch_jd, r_ecef, v_ecef)
# sv_eci = sv_ecef_to_eci(sv_ecef, ITRF(), GCRF(), eop_data)
# println(sv_eci.r)